# Vergelijk energieverbruik recente dagen

In [1]:
import io
import os
import sys
import types
import numpy as np
import pandas as pd
import altair as alt

In [2]:
%run MakeNBImportAvailable.ipynb

In [3]:
from datetime import date
from datetime import datetime, timedelta
# import RetrieveData as rd_gn
# import knmi_gn_001 as gethdd
# import Process_Data_CH as prcs
import GetMySQL

importing Jupyter notebook from GetMySQL.ipynb


In [4]:
# settings = GetMySQL.readsettings()

In [5]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [6]:
def setpoi(nrdays):
    today = date.today()
    startday = today -timedelta(days=nrdays)
    poi = pd.date_range(startday,today)
    return poi.strftime('%Y-%m-%d')
# nrdays = 4
# days_poi = (date.today()- timedelta( days= nrdays)).strftime('%Y-%m-%d')
# days_poi = ''.join([days_poi,':'])
# days_poi = GetMySQL.SetPOI(days_poi)



In [7]:


# days_poi = ['2022-05-07','2022-05-08','2022-05-09','2022-05-10']
# days_poi = ['2022-05-03','2022-05-04']

In [8]:
def readdata(poi,props):
    df = []
    for d in days_poi:
        poi = pd.to_datetime(d)+timedelta(days=1)
        poi = poi.strftime('%Y-%m-%d')
        poi = ''.join([d,':',poi])
        poi = GetMySQL.SetPOI(poi)

        df_g = GetMySQL.getSQL(poi,"item0054",props,"gas")
        df_g = GetMySQL.RemoveReadingErrors(df_g)
        df_EH = GetMySQL.getSQL(poi,"item0052",props,"EH")
        df_EH = GetMySQL.RemoveReadingErrors(df_EH)
        df_EL = GetMySQL.getSQL(poi,"item0051",props,"EL")
        df_EL = GetMySQL.RemoveReadingErrors(df_EL)
        df_ = pd.concat([df_g,df_EH,df_EL])
        df.append(df_)

    df = pd.concat(df)
    return(df)

In [9]:
def cominbeerEHEL(df):
    # Electricity in 1 table
    table = pd.pivot_table(df[df['variable']!='gas'], values='verbruik', index=['datetime','dy'],
                        columns=['variable'], aggfunc=np.sum)
    table['variable']='E'
    table['verbruik']=table['EH'] + table['EL']
    table=table.reset_index()
    return(table)
    

In [10]:
def plotdata(df):
    wdth = 300
    source = df
    chart2a = alt.Chart(df[df['variable']!='gas'],
                               title='elektriciteit cumulatief').mark_line(
            strokeWidth=1).encode(
            x=alt.X('normalday:T',
                axis=alt.Axis(title="tijd")),
            y=alt.Y("cumul:Q",
                axis=alt.Axis(title="verbruik [kWh]")),
        color=alt.Color('dy:N')
        ).properties(width=wdth,height=250).interactive()


    chart2b = alt.Chart(df[df['variable']=='gas'],
                        title='gas cumulatief').mark_line(
            strokeWidth=1).encode(
            x=alt.X('normalday:T'),
            y=alt.Y("cumul:Q",
                axis=alt.Axis(title="verbruik [m3]")),
        color=alt.Color('dy:N')
        ).properties(width=wdth,height=250).interactive()

    chart_all=alt.vconcat(chart2a|chart2b)
    display(chart_all)
    

In [11]:
def laatmaarzien(poi):
    props = GetMySQL.readsettings()
    df = readdata(poi,props)
    df = df.reset_index()
    # verbruik
    df['verbruik'] = df[['counter','variable','dy']].groupby(['variable','dy']).diff().shift(-1)
    df = df.fillna(0)
    
    # combineer electriciteit Hoog & Laag tarief
    df_E = cominbeerEHEL(df)
    df_g = df[df['variable']=='gas']
    cols = ['datetime','variable','verbruik','dy']
    df = pd.concat([df_g[cols],df_E[cols]])
    
    df['cumul'] = df[['variable','verbruik','dy']].groupby(['variable','dy']).cumsum()
    df['dy'] = df['dy'].astype(str)
    df['normalday'] = '2022-01-01'
    df['normalday'] = df['normalday'] +' '+ df['datetime'].dt.hour.astype(str)+':'+df['datetime'].dt.minute.astype(str)
    df['normalday'] = pd.to_datetime(df['normalday'])
    
    plotdata(df)
    
    return(df)

In [12]:
days_poi = setpoi(5)
df = laatmaarzien(days_poi)

POI is :  {'start': '2022-10-09', 'end': '2022-10-10'}
POI is :  {'start': '2022-10-10', 'end': '2022-10-11'}
POI is :  {'start': '2022-10-11', 'end': '2022-10-12'}
POI is :  {'start': '2022-10-12', 'end': '2022-10-13'}
POI is :  {'start': '2022-10-13', 'end': '2022-10-14'}
POI is :  {'start': '2022-10-14', 'end': '2022-10-15'}


alt.VConcatChart(...)

In [13]:
df.head()

,datetime,variable,verbruik,dy,cumul,normalday
0,2022-10-04 00:00:00,gas,0.0,2022-10-04,0.0,2022-01-01 00:00:00
1,2022-10-04 00:01:00,gas,0.0,2022-10-04,0.0,2022-01-01 00:01:00
2,2022-10-04 00:02:00,gas,0.0,2022-10-04,0.0,2022-01-01 00:02:00
3,2022-10-04 00:03:00,gas,0.0,2022-10-04,0.0,2022-01-01 00:03:00
4,2022-10-04 00:04:00,gas,0.0,2022-10-04,0.0,2022-01-01 00:04:00


In [14]:
def calc_uur(df):
    df = df.groupby([pd.Grouper(key='datetime', freq='h'), 'variable','dy']).verbruik.sum().reset_index()
    
    df['normalday'] = '2022-01-01'
    df['normalday'] = df['normalday'] +' '+ df['datetime'].dt.hour.astype(str)+':' + df['datetime'].dt.minute.astype(str)
    df['normalday'] = pd.to_datetime(df['normalday'])
    # df['dy'] = 
   
    return df

In [32]:
df_h = calc_uur(df)
df_h.tail()

,datetime,variable,dy,verbruik,normalday
281,2022-10-11 20:00:00,gas,2022-10-11,0.000,2022-01-01 20:00:00
282,2022-10-11 21:00:00,E,2022-10-11,0.347,2022-01-01 21:00:00
283,2022-10-11 21:00:00,gas,2022-10-11,0.000,2022-01-01 21:00:00
284,2022-10-11 22:00:00,E,2022-10-11,0.131,2022-01-01 22:00:00
285,2022-10-11 22:00:00,gas,2022-10-11,0.000,2022-01-01 22:00:00


In [33]:
def plot_hr_data(df):
    wdth = 600
    source = df
    chart2a = alt.Chart(df[df['variable']!='gas'],
                               title='elektriciteit door de dag').mark_line(
            strokeWidth=1).encode(
            x=alt.X('normalday:T',
                axis=alt.Axis(title="tijd")),
            y=alt.Y("verbruik:Q",
                axis=alt.Axis(title="verbruik [kWh]")),
        color=alt.Color('dy:N')
        ).properties(width=wdth,height=250).interactive()
    display(chart2a)

In [34]:
plot_hr_data(df_h)

alt.Chart(...)